# Archivo de pruebas para CDS de base de riesgo para la nueva lógica

In [1]:
import pandas as pd
import requests
import logging
import base64
import json

class CoreDataService:

        """
        Clase para conectar y recuperar datos de SAP CDS views.

        Attributes:
            base_url (str): URL base para la conexión con la API.
            batch_mode (bool): Indica si las consultas deben ser procesadas en lotes.
            batch_size (int): Tamaño de cada lote si batch_mode es True.
        """

        def __init__(self, base_url, batch_mode=True, batch_size=150):

            """
            Inicializa la instancia con la configuración de conexión necesaria.

            Args:
                base_url (str): URL base para la conexión con la API.
                batch_mode (bool): Habilita o deshabilita la consulta en lotes.
                batch_size (int): Define el tamaño de cada lote para la consulta.
            """

            self.base_url = base_url
            self.session = requests.Session()
            self.batch_mode = batch_mode
            self.batch_size = batch_size
            logging.info(f"CoreDataService initialized with base URL: {base_url}")

        def query_cds(self, materials, field, from_cds, column_name):

            """
            Consulta datos según la configuración de batch_mode y maneja errores por lote o por material.

            Args:
                materials (list): Lista de materiales para consulta.
                field (str): Campo seleccionado para el query.
                from_cds (str): Nombre de la vista CDS desde donde se consulta.
                column_name (str): Nombre de la columna para el filtro.

            Returns:
                tuple: Un DataFrame con todos los resultados concatenados y un diccionario de errores.
            """

            results = []
            errors = {}

            if self.batch_mode:
                for i in range(0, len(materials), self.batch_size):
                    batch = materials[i:i + self.batch_size]
                    result, batch_errors = self.process_batch(batch, field, from_cds, column_name)
                    if result is not None:
                        results.append(result)
                    if batch_errors:
                        errors.update(batch_errors)
            else:
                for material in materials:
                    try:

                        result = self.process_individual(material, field, from_cds, column_name)
                        if result is not None and not result.empty:
                            results.append(result)

                    except Exception as e:
                        logging.error(f"Error processing material {material}: {str(e)}")
                        errors[material] = str(e)
            return pd.concat(results, ignore_index=True) if results else pd.DataFrame(), errors

        def process_batch(self, batch, field, from_cds, column_name):

            """
            Procesa un lote de materiales y devuelve los resultados y errores.
            """

            json_query = self.build_query(batch, field, from_cds, column_name)

            try:

                # Asumiendo que transform_json_input_to_df solo devuelve un DataFrame y no una tupla
                result = self.transform_json_input_to_df(json_query)
                return result, {}  # Devuelve un diccionario vacío de errores si todo sale bien

            except Exception as e:
                logging.error(f"Error processing batch: {str(e)}")
                # Devuelve un DataFrame vacío y un diccionario con los errores correspondientes al lote
                return None, {mat: str(e) for mat in batch}

        def process_individual(self, material, field, from_cds, column_name):

            """
            Procesa un único material y devuelve el resultado.
            """

            json_query = self.build_query([material], field, from_cds, column_name)
            return self.transform_json_input_to_df(json_query)

        def transform_json_input_to_df(self, json_data):

            """
            Envía un query JSON a la API y retorna un DataFrame.
            """

            base64_data = base64.b64encode(json.dumps(json_data).encode()).decode()
            url = f'{self.base_url}{base64_data}'
            response = self.session.get(url)
            response.raise_for_status()
            return pd.DataFrame(response.json())

        def build_query(self, materials, field, from_cds, column_name):

            """
            Construye el query JSON para la API.
            """

            filter_query = " or ".join([f"{column_name} eq '{material}'" for material in materials])
            return {"FIELD": field, "FROM": from_cds, "WHERE": filter_query}


In [2]:
service = CoreDataService("http://sapapp01.prebel.com.co:8000/services/zws_gl_0001?SELECT=", batch_mode=True)
data, errors = service.query_cds("*", "*", "ZMMT010ZMMT0072", "*")

In [3]:
df = pd.DataFrame(data)
df.head()

,MANDT,ERDAT,MATNR,MAKTX,MEINS,MAABC,MTART,MARCA,ZNEGOC,LABST,...,METASKU,METAABC,METAMARCA,EXCEMES,STOCKID,EXCESUN,EXCESVR,STATUS,COBERTURA,WAERS
0,400,2025-06-30,PB0046459,DEL LIQU OJOS VITU SABILA DUO NEGX1.5ML,ST,A,ZMER,017,RETAIL,3416.0,...,0.0,1.0,5.0,15,209.0,3207.0,209994.36,R,5.Mayor o igual a 360 dias,COP
1,400,2025-05-31,PB0103727,SERUM REVOX R RETINOL REGENERA. 30 ML,ST,B,ZMER,114,RETAIL,4137.0,...,0.0,1.0,0.0,7,0.0,4137.0,315859.95,R,3.Entre 180 y 270 dias,COP
2,400,2025-05-31,PB0103740,BP FAM TUBE HAIR GEL 100 GR,ST,,VERP,015,FPT,814.0,...,0.0,0.0,3.0,0,6582.2,0.0,0.00,V,1.Menor de 90 dias,COP
3,400,2025-05-31,PB0103756,KP ME+ 60ML DEEP BR 6/77,ST,B,ZMER,011,RETAIL,305.0,...,0.0,1.0,4.0,8,98.8,206.2,13685.49,R,5.Mayor o igual a 360 dias,COP
4,400,2025-05-31,PB0103763,TAP MEC SOF TOR SIN OVA PP VITAL 62G,ST,B,VERP,079,FPT,22182.0,...,0.0,1.0,0.0,0,0.0,22182.0,42811.26,V,1.Menor de 90 dias,COP


In [4]:

df_cds_final = data.drop(columns=[
    'MANDT', 'MEINS', 'MAABC', 'MTART', 'MARCA',
    'ZNEGOC', 'PROMVC', 'MESINV', 'DIAINV', 'METASKU',
    'METAABC', 'METAMARCA', 'EXCEMES', 'STOCKID', 'EXCESUN',
    'EXCESVR', 'STATUS', 'COBERTURA', 'WAERS', 'REAL3', 'REAL2',
    'VR_LABST'
])

df_cds_final = df_cds_final.rename(columns={
    'ERDAT': 'FECHA',
    'MATNR': 'MATERIAL',
    'MAKTX': 'DESCRIPCION',
    'LABST': 'LIBRE UTILIZACION'
})

df_cds_final.head(10)

,FECHA,MATERIAL,DESCRIPCION,LIBRE UTILIZACION,REAL1,ESTIM1,ESTIM2,ESTIM3
0,2025-06-30,PB0046459,DEL LIQU OJOS VITU SABILA DUO NEGX1.5ML,3416.0,197.0,197.0,142.0,131.0
1,2025-05-31,PB0103727,SERUM REVOX R RETINOL REGENERA. 30 ML,4137.0,686.0,660.0,993.0,0.0
2,2025-05-31,PB0103740,BP FAM TUBE HAIR GEL 100 GR,814.0,0.0,0.0,3120.0,0.0
3,2025-05-31,PB0103756,KP ME+ 60ML DEEP BR 6/77,305.0,24.0,43.0,43.0,0.0
4,2025-05-31,PB0103763,TAP MEC SOF TOR SIN OVA PP VITAL 62G,22182.0,0.0,0.0,0.0,0.0
5,2025-05-31,PB0103781,JAR LAB HIDRA DOVE TC GEN 300G DEMAIS,10472.0,0.0,0.0,59596.0,0.0
6,2025-05-31,PB0103784,JAR LAB REGEN DOVE TC GEN 300G DEMAIS,5232.0,0.0,0.0,29852.0,0.0
7,2025-05-31,PB0103812,KP ME+ 60ML DEEP BR 5/77,125.0,13.0,18.0,18.0,0.0
8,2025-05-31,PB0103823,KP ME+ 60ML PURNAT 55/0,715.0,21.0,28.0,30.0,0.0
9,2025-05-31,PB0103842,KP ME+ 60ML RCHNAT 6/91,72.0,0.0,5.0,5.0,0.0


### Luego de organizar la información aplicamos la nueva lógica

In [ ]:
def calculate_rango_cobertura_column(row: pd.Series) -> str:
    """
    Calcula el rango de cobertura aplicando la nueva lógica

    Args:
        row (pd.Series): Fila de datos

    Returns:
        str: Rango de cobertura con la nueva lógica
    """

    days = row.get("DIAS COBERTURA")

    if days <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < days <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < days <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < days <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < days <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < days <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"    

df_cds_final["RANGO COBERTURA"] = df_cds_final.apply(calculate_rango_cobertura_column, axis=1)